<a href="https://colab.research.google.com/github/Ignacio-Ibarra/NLP-Disasters/blob/master/Real_or_Not%3F_NLP_with_Disaster_Tweets_Informe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Real or Not? NLP with Disaster Tweets**

### **Autores**: Alejandra Toscano - Diego Sanchez - Erik Cirilo - Ignacio Ibarra

## **Introducción** 

Esta es un trabajo práctico realizado para la materia 7506 - Organización de Datos de la Facultad de Ingniería de la Universidad de Buenos Aires. Se trata de una competencia lanzada en la plataforma Kaggle (ver [aquí](https://www.kaggle.com/c/nlp-getting-started)), la cual consiste en predecir si un tweet nos está hablando de una desastre o no, implementando técnicas de _natural language processing_ (NLP).   

Para ello contamos con un _dataset_ de entrenamiento y otro para testear nuestro modelo de _machine learning._ 

Este trabajo consta de dos entregas: 

* **Primera parte: Análisis Exploratorio de los Datos**

>El principal objetivo de esta parte es hallar algunos _insights_ mediante visualizaciones de las distintas relaciones entre las variables. Este trabajo se realizará sobre el _dataset_ de entrenamiento. Las variables presentes en el mismo son: 

  >- `id` : una identificación única de cada una de las observaciones
  
  >-  `keyword`: es una palabra que el usuario de Twitter puede cargar al momento de postear un tweet con el objetivo de asociar el texto a dicha palabra o concepto. Podría ser totalmente arbitraria la elección del _keyword_

  >- `location`: el usuario también puede cargar la locación desde dónde realiza el posteo. También es arbitraria dicha elección porque no se carga desde la locación del GPS. 

  >- `text`: es el texto del tweet. 

  >- `target`: es una variable binaria que toma valor 1 cuando el tweet corresponde con un desastre y 0 cuando no. 

* **Segunda parte: Modelo de machine learning basado en NLP** 

>El objetivo de esta segunda parte es desarrollar un modelo que luego de ser probado en la base de entrenamiento podamos utilizarlo para predecir la variable target. 

## **Pre-procesamiento**



### **Limpieza del dataset** 



#### **Limpieza variable `text`**

Para trabajar con más claridad con esta variable hemos eliminado las url, los _hashtags_, las _mentions_ del texto de cada tweet, esto nos permitirá luego hacer un mejor análisis sobre los "duplicados". 

#### **Registros duplicados**

En primer lugar comprobamos que dado que el id es una clave única no hay filas duplicadas. 

Sin embargo, lo que cuenta es definir también un criterio para contabilizar valores como 'duplicados'. Para ello hemos redefinido la variable `text` como una variable que alberga sólo oraciones (sin url, _hashtags_, _mentions_)  y hemos considerado los repetidos sin tener en cuenta el `id`. No se consideraron diferencias en la puntuación.  
 
De esta manera comprobamos que para un mismo texto había dos o más filas que tenían distinto valor de `target`. Como se ve en la siguiente tabla, la columna 'number_of_uniques' indica cuántos valores distintos en la columna `target` tiene dicho texto, viendo sólo los primeros 15 casos ordenados por la cantidad de repeticiones de manera descendente. 

clean_text|count|number_of_uniques
---|:---:|:---:|		
watch this airport get swallowed up by a sandstorm in under a minute |24|	1
wreckage 'conclusively confirmed' as from mh370: malaysia pm: investigators and the families of those who were...	|20|	1
families to sue over legionnaires: more than 40 families affected by the fatal outbreak of legionnaires' disea...	|19|	1
funtenna: hijacking computers to send data as sound waves [black hat 2015]	|17|	2
christian attacked by muslims at the temple mount after waving israeli flag via pamela geller - ...	|15|	1
obama declares disaster for typhoon-devastated saipan: obama signs disaster declaration for northern marians a...	|13	|1
the latest: more homes razed by northern california wildfire - abc news	|13|	1
11-year-old boy charged with manslaughter of toddler: report: an 11-year-old boy has been charged with manslaughter over the fatal sh...	|10|	1
70 years after atomic bombs japan still struggles with war past: the anniversary of the devastation wrought b...	|9|	1
refugio oil spill may have been costlier bigger than projected	|9|	1
the latest: more homes razed by northern california wildfire -	|9|	1
c-130 specially modified to land in a stadium and rescue hostages in iran in 1980	|8|	2
u.s national park services tonto national forest: stop the annihilation of the salt river wild horse... via	|8|	2
twelve feared killed in pakistani air ambulance helicopter crash	|8|	1
rly tragedy in mp: some live to recount horror: ûïwhen i saw coaches of my train plunging into water i called ...	|8|	1


Que la variable `text` tenga repetidos puede ser por múltiples razones. Puede deberse a un mal trabajo de data-entry. También puede que haya distintos usuarios copiando y pegando el mismo texto. Es decir, no es del todo claro si corresponde borrarlos o no, básicamente porque no conocemos bien cómo se ha armando el _train set_. 

Dado que los textos repetidos pueden llegar a ser un problema en nuestro modelo: tendremos que tener en consideración dos modelos uno que trabaje con estos textos duplicados y otro que trabaje sin los textos duplicados.

No quedan dudas con respecto a los registros que tienen `target` contradictorio. Esto fueron eliminados, pasando de tener 7613 a 7364 filas. 

#### **Tratamiento _null values_**

Por otro lado, hemos reemplazado los _null values_ de las columnas `keyword` (61) y `location` (2533) por 'no_keyword' y 'no_location' respectivamente, para no perder ni filas ni columnas. 

## **Proporción de Target Real y No Real**

La variable `target` está distribuida de la siguiente forma en el _dataset_ de entrenamiento. 

![Bar Chart - Cantidad de Tweets por Target](https://drive.google.com/uc?id=1xIAFSh3PyCGB1Z0pRFbykBULTl8IyJFH)

![Pie Chart - Porcentaje de Tweets por Target](https://drive.google.com/uc?id=1r7NAc0aH19QOXHxXlwMrrMaO3Z2cl1ng)


### **Analizando `keyword`**

La palabra clave es cargada por el usuario de Twitter al momento del post. Sirve para relacionar un tweet con un concepto y facilita que el motor de búsqueda de Twitter devuelva dicho post si alguien realiza una búsqueda utilizando dicha palabra clave. 

Si bien la elección del 'keyword' por parte del usuario puede guardar algún tipo de relación con el texto del tweet, no se puede asegurar que eso suceda ya que dicha elección es a _priori_ arbitraria. 

La variable en análisis puede presentarse como una única palabra o como una conjunción de más de una. 

Los _unique values_ son el 2.92% del total de registros de dicha variable. En la siguiente tabla podemos ver las palabras que más se repiten y las que menos, sin tomar en cuenta el valor 'no_keyword'.  

keyword|Frecuencia absoluta
---|---|
fatalities |              45
deluge      |             42
armageddon   |            42
harm          |           41
..|                         ..
forest%20fire     |       19
epicentre     |           12
threat        |           11
inundation    |           10
radiation%20emergency |    9

A su vez, el valor 'no_keyword' aparece 61 veces en la muestra, pero a los fines de saber cuáles son las palabras o conceptos que los tweets están volcando, el valor 'no_keyword' no aporta información valiosa. 

#### **`keyword` por `target`**

Agrupando por `keyword` obtenemos la proporción de registros que tienen un valor de `target` igual a 1 y calculamos el porcentaje de "veracidad" sobre el total de registros para ese valor de `keyword`. El porcentaje de veracidad no es más que la media por 100. De las 222 keywords hemos seleccionado las 25 con porcentaje más alto y las 25 con porcentaje más bajo. 

![Horizontal Bar Chart: Top 25 Keywords Frecuentes por Target](https://drive.google.com/uc?id=1dguMuchm5IN8vV3JwbW5sCAVJTevI8p4)

En la figura de la izquierda, se observan keywords referentes a desastres naturales (_drought, forest fire, violent storm) o edilicios (bridge collapse, buildings on fire_). También se encuentran tragedias mortales (_mass murder, airplane accident, nuclear disaster, suicide bombing_). Otros keywords incluyen lenguaje de noticias (_casualties, evacuated, rescuers_).

En la figura de la derecha, su mayoría se trata de palabras que servirían para describir tragedias pero que a su vez pueden usarse en muchos otros contextos. Un ejemplo de esto es el keyword "body bag", con un porcentaje de targets Reales menor al 5%. El término por si solo sonaría alarmante, pero veamos las acepciones de la palabra:

1.   Bolsa para guardar cadáveres
2.   Una clase de cartera/mochila
3.   Según *Urban Dictionary*, "To boddybag an oponent" es asegurar la victoria sobre el oponente (algo así como "kick ass"), es un slang muy utilizado en las batallas de rap.
4.   Otra acepción de *Urban Dictionary*, "Bodybagging" es robarle un chiste a otro comediante y hacerlo pasar como propio. 

#### **¿Hay alguna relación entre la frecuencia del `keyword` y el porcentaje de veracidad?** 

Nos preguntamos si las keyword que tienen un porcentaje de veracidad más elevado tienen a su vez una frecuencia más alta. 

Observamos que la relación entre la cantidad de apariciones de un `keyword` y el porcentage de veracidad de dicho keyword es débil. Como lo muestra la dispersión de puntos de manera horizontal. 

![Sctatter plot: Relación entre el porcentaje de veracidad y frecuencia del `keyword` ](https://drive.google.com/uc?id=13CcSoLQG1Hwu4ToR9vZcj5x3eWAab9g5)

Lo mismo muestra la matriz de correlaciones, con valores cercanos a cero. 

corr|percentage|	count
---|---|---
percentage|	1.00|	0.08
count	|0.08|	1.00

#### **¿Hay keywords distintas para un mismo texto?**

Agrupando los datos por `clean_text` (variable que creamos para analizar los duplicados) vimos que existen 29 casos en los que hay dos `keyword` distintas por cada `texto`. Esto implicaría o bien que hay conexiones entre las `keyword` o bien que son errores de data-entry. (FALTA DESCRIBIR CUALES SON LAS KEYWORDS CONECTADAS) 





### **Analizando `location`**

#### **Distribución**




